# Externally Computed Polars

This example shows how to build, using the level-2 interface, an aviary model that includes an external susbsystem that computes a lift and drag polar and passes them into the mission aerodynamics for a 3-phase mission (climb, cruise, descent). During the mission, Aviary will interpolate on the computed polars to compute actual lift and drag for a given flight condition.

We start with the assumption that we have an external component called `ExternalAero` that can compute the lift and drag at any given altitude, mach number, and angle of attack. The details of such a component may be highly complicated and not important for the purposes of this example. We will be using a structured grid, which assumes the data table is regularly spaced in all dimensions. We want to compute lift and drag over a grid of altitudes (in 'ft'), mach numbers, and angles of attack given by:

In [ ]:
# This hidden cell just creates a component that produces the lift and drag polar. 
# The implementation details are unimportant to the example.
import openmdao.api as om
import numpy as np

from aviary.api import Aircraft
import aviary.api as av

# The drag-polar-generating component reads this in, instead of computing the polars.
polar_file = "subsystems/aerodynamics/gasp_based/data/large_single_aisle_1_aero_free_reduced_alpha.txt"

aero_data = av.read_data_file(polar_file, aliases = {'altitude': 'altitude',
                                                  'mach': 'mach',
                                                  'angle_of_attack': 'angle_of_attack',
                                                  'lift_coefficient': 'cl',
                                                  'drag_coefficient': 'cd'})

altitude = np.unique(aero_data.get_val('altitude', 'ft'))
mach = np.unique(aero_data.get_val('mach', 'unitless'))
angle_of_attack = np.unique(aero_data.get_val('angle_of_attack', 'deg'))

shape = (altitude.size, mach.size, angle_of_attack.size)
CL = aero_data.get_val('lift_coefficient').reshape(shape)
CD = aero_data.get_val('drag_coefficient').reshape(shape)


class ExternalAero(om.ExplicitComponent):
    """
    This component is a stand-in for an externally computed lift/drag table
    calculation. It does nothing but read in the pre-computed table. A real
    component would actually computed the values at all requested points.
    """
    def initialize(self):
        """
        Declare options.
        """
        self.options.declare("altitude", default=None, allow_none=True,
                             desc="List of altitudes in ascending order.")
        self.options.declare("mach", default=None, allow_none=True,
                             desc="List of mach numbers in ascending order.")
        self.options.declare("angle_of_attack", default=None, allow_none=True,
                             desc="List of angles of attack in ascending order.")

    def setup(self):
        altitude = self.options['altitude']
        mach = self.options['mach']
        angle_of_attack = self.options['angle_of_attack']

        self.add_input(Aircraft.Wing.AREA, 1.0, units='ft**2')
        self.add_input(Aircraft.Wing.SPAN, 1.0, units='ft')

        shape = (len(altitude), len(mach), len(angle_of_attack))

        self.add_output('drag_table', shape=shape, units='unitless')
        self.add_output('lift_table', shape=shape, units='unitless')

    def compute(self, inputs, outputs):
        """
        This component doesn't do anything, except set the drag and lift
        polars from the file we read in.
        """
        outputs['drag_table'] = CD
        outputs['lift_table'] = CL
        
print('Altitude (ft)', altitude)
print('Mach', mach)
print('Angle of Attack (deg)', angle_of_attack)

In a structured grid, interpolation data must be present for every combination of inputs. In other words, our `ExternalAero` component must run a full factorial of points spanning those 3 variables. The Aviary variable hierarchy includes two variables for the polars: `Aircraft.Design.LIFT_POLAR`, and `Aircraft.Design.DRAG_POLAR`. The data in each of these polars should be a `n` x `m` x `k` numpy array, where `n` is the number of altitudes, `m` is the number of mach numbers, and `k` is the number of angles of attack. The `ExternalAero` will need to compute these values and place them into an array of this shape.

If use of a structured grid is not desirable, then the data does not need to meet these formatting requirements. In that case, the data table does not have to be regularly spaced, and each variable (`Altitude`, `Mach`, `angle_of_attack`, `LIFT_POLAR`, and `DRAG_POLAR`) must be 1-dimensional numpy arrays of equal length.

Using the level-2 interface, we create a builder for our external `ExternalAero` subsystem. In this example, the component produces outputs `drag_table` and `lift_table`, but we can specify an alias to `Aircraft.Design.DRAG_POLAR` and `Aircraft.Design.LIFT_POLAR` respectively. It is important that we inherit from the `AerodynamicsBuilderBase` to let Aviary know this is builder produces aerodynamics components. Some mission analysis methods require special handling of aerodynamics components that will not occur if we skip this step.

In [ ]:
class ExternalAeroBuilder(av.AerodynamicsBuilderBase):
    """
    An example subsystem builder that adds an external aerodynamics component

    Parameters
    ----------
    aero_data : NamedValues
        Altitude, Mach number, and angle of attack data, all in ascending order.
    """

    def __init__(self, name='aero', altitude=None, mach=None,
                 angle_of_attack=None):
        super().__init__(name)
        self.altitude = altitude
        self.mach = mach
        self.angle_of_attack = angle_of_attack
        
    def build_pre_mission(self, aviary_inputs):
        """
        Build an OpenMDAO system for the pre-mission computations of the subsystem.

        Returns
        -------
        pre_mission_sys : openmdao.core.Group
            An OpenMDAO group containing all computations that need to happen in
            the pre-mission part of the Aviary problem. This includes sizing, design,
            and other non-mission parameters.
        """

        return ExternalAero(altitude=self.altitude, mach=self.mach, angle_of_attack=self.angle_of_attack)

    def mission_outputs(self):
        return [('drag_table', Aircraft.Design.DRAG_POLAR),
                ('lift_table', Aircraft.Design.LIFT_POLAR)]

Notice that we have passed the altitude, Mach, and angle of attack arrays into the builder so that the ExternalAero component can use them as instantiation arguments.

Next, we add the builder to our phase_info as usual. We are using a single-aisle commercial transport aircraft and mission.

In [ ]:
phase_info = av.default_height_energy_phase_info.copy()

external_aero = ExternalAeroBuilder(name='external_aero',
                                    altitude=altitude, mach=mach, angle_of_attack=angle_of_attack)

phase_info['pre_mission']['external_subsystems'] = [external_aero]

Next, the existing mission phases need to be given the information they need to set up our aerodynamics analysis using `phase_info`. We use the `solved_alpha` method of Aviary's included aerodynamics for this, which can accept the input passed from our external subsystem. Since we are using Aviary's built-in aerodynamics methods, we use the default name "core_aerodynamics". Don't forget to update the `subsystem_options` for each phase. We must specify the `method`, the `aero_data` that contains our altitude, Mach, and angle of attack data, as well as the `training_data` flag to denote we are passing our drag polars via openMDAO connections.

In [ ]:
subsystem_options = {'method': 'solved_alpha',
                     'aero_data': aero_data,
                     'training_data': True}

phase_info['climb']['subsystem_options'] = {'core_aerodynamics': subsystem_options}
phase_info['cruise']['subsystem_options'] = {'core_aerodynamics': subsystem_options}
phase_info['descent']['subsystem_options'] = {'core_aerodynamics': subsystem_options}


Finally, we can instantiate the AviaryProblem like normal. However, we need to tell Aviary the size of our lift and drag polars so that it can allocate the right shape for the connection.

In [ ]:
import pkg_resources

from aviary.api import AviaryProblem

input_file = pkg_resources.resource_filename(
    "aviary", "models/test_aircraft/aircraft_for_bench_GwFm.csv")

prob = av.AviaryProblem()
prob.load_inputs(input_file, phase_info)

# Preprocess inputs
prob.check_and_preprocess_inputs()

# Add correctly-sized polar to aviary_inputs so that input components are sized correctly.
shape = (altitude.size, mach.size, angle_of_attack.size)

prob.aviary_inputs.set_val(Aircraft.Design.LIFT_POLAR, np.zeros(shape), units='unitless')
prob.aviary_inputs.set_val(Aircraft.Design.DRAG_POLAR, np.zeros(shape), units='unitless')


In [ ]:
# get through setup for the problem
prob.add_pre_mission_systems()
prob.add_phases()
prob.add_post_mission_systems()
prob.link_phases()
prob.add_driver("SLSQP")
prob.add_design_variables()
prob.add_objective(objective_type="mass", ref=-1e5)
prob.setup()